In [2]:
data <- read.csv('binary.csv')
head(data)

,admit,gre,gpa,rank
,<int>,<int>,<dbl>,<int>
1,0,380,3.61,3
2,1,660,3.67,3
3,1,800,4.00,1
4,1,640,3.19,4
5,0,520,2.93,4
6,1,760,3.00,2


In [4]:
str(data)

'data.frame':	400 obs. of  4 variables:
 $ admit: int  0 1 1 1 0 1 1 0 1 0 ...
 $ gre  : int  380 660 800 640 520 760 560 400 540 700 ...
 $ gpa  : num  3.61 3.67 4 3.19 2.93 3 2.98 3.08 3.39 3.92 ...
 $ rank : int  3 3 1 4 4 2 1 2 3 2 ...


In [5]:
data$rank <- factor(data$rank)
data$admit <- factor(data$admit)
str(data)


'data.frame':	400 obs. of  4 variables:
 $ admit: Factor w/ 2 levels "0","1": 1 2 2 2 1 2 2 1 2 1 ...
 $ gre  : int  380 660 800 640 520 760 560 400 540 700 ...
 $ gpa  : num  3.61 3.67 4 3.19 2.93 3 2.98 3.08 3.39 3.92 ...
 $ rank : Factor w/ 4 levels "1","2","3","4": 3 3 1 4 4 2 1 2 3 2 ...


In [7]:
xtabs(~admit + rank , data = data)

     rank
admit  1  2  3  4
    0 28 97 93 55
    1 33 54 28 12

In [11]:
set.seed(1234)
ind <- sample(2 , nrow(data) , prob = c(0.8,0.2) , replace = TRUE)
tdata <- data[ind == 1, ]
vdata <- data[ind == 2 , ]
print(dim(tdata))
print(dim(vdata))

[1] 325   4
[1] 75  4


In [12]:
#Logistic Regreesion Model 
mymodel <- glm(admit~. , data = tdata , family = 'binomial')

In [14]:
summary(mymodel)


Call:
glm(formula = admit ~ ., family = "binomial", data = tdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5873  -0.8679  -0.6181   1.1301   2.1178  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -5.009514   1.316514  -3.805 0.000142 ***
gre          0.001631   0.001217   1.340 0.180180    
gpa          1.166408   0.388899   2.999 0.002706 ** 
rank2       -0.570976   0.358273  -1.594 0.111005    
rank3       -1.125341   0.383372  -2.935 0.003331 ** 
rank4       -1.532942   0.477377  -3.211 0.001322 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 404.39  on 324  degrees of freedom
Residual deviance: 369.99  on 319  degrees of freedom
AIC: 381.99

Number of Fisher Scoring iterations: 4


In [17]:
mymodel1 <- glm(admit ~ gpa + rank , data = tdata , family = 'binomial')
summary(mymodel1)


Call:
glm(formula = admit ~ gpa + rank, family = "binomial", data = tdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5156  -0.8880  -0.6318   1.1091   2.1688  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -4.7270     1.2918  -3.659 0.000253 ***
gpa           1.3735     0.3590   3.826 0.000130 ***
rank2        -0.5712     0.3564  -1.603 0.108976    
rank3        -1.1645     0.3804  -3.061 0.002203 ** 
rank4        -1.5642     0.4756  -3.289 0.001005 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 404.39  on 324  degrees of freedom
Residual deviance: 371.81  on 320  degrees of freedom
AIC: 381.81

Number of Fisher Scoring iterations: 4


In [24]:
pred <- predict(mymodel1 , tdata , type = 'response')

In [26]:
print(head(pred))
head(tdata)

        1         2         3         4         6         7 
0.2822956 0.2992879 0.6828897 0.1290134 0.2354735 0.3466234 


,admit,gre,gpa,rank
,<fct>,<int>,<dbl>,<fct>
1,0,380,3.61,3
2,1,660,3.67,3
3,1,800,4.00,1
4,1,640,3.19,4
6,1,760,3.00,2
7,1,560,2.98,1


In [33]:
#Error for training data
pred1 <- ifelse(pred>0.5 , 1 , 0)
tab <- table(predicted =pred1 , actual = tdata$admit)
tab
1 - sum(diag(tab)) / sum(tab)

         actual
predicted   0   1
        0 208  73
        1  15  29

[1] 0.2707692

In [36]:
#Predicting test data
pred2 <- predict(mymodel1 , vdata , type = 'response')
pred3 <- ifelse(pred2 > 0.5 , 1 , 0)
tab1 <- table(pred3 , vdata$admit)
tab1
1 - sum(diag(tab1)) / sum(tab1)

     
pred3  0  1
    0 48 20
    1  2  5

[1] 0.2933333

In [38]:
#Goodness of Fit
with(mymodel1 , pchisq(null.deviance - deviance , df.null - df.residual , lower.tail = F))

[1] 1.450537e-06

Since the p value is so  small we can say that the model is statistically significant